In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import json
import os
import requests
import io
import zipfile

/tmp/ipykernel_56920/2038222792.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
table_names = {
  'MB_2021_AUST': 'Mesh Blocks - 2021',
  'SA1_2021_AUST': 'Statistical Areas Level 1 - 2021',
  'SA2_2021_AUST': 'Statistical Areas Level 2 - 2021',
  'SA3_2021_AUST': 'Statistical Areas Level 3 - 2021',
  'SA4_2021_AUST': 'Statistical Areas Level 4 - 2021',
  'GCCSA_2021_AUST': 'Greater Capital City Statistical Areas - 2021',
  'STE_2021_AUST': 'States and Territories - 2021',
  'AUS_2021_AUST': 'Australia - 2021',
  'INDIGENOUS_STRUCTURE_ALLOCATION_2021': 'Indigenous Structure - 2021',
  'ILOC_2021_AUST': 'Indigenous Locations - 2021',
  'IARE_2021_AUST': 'Indigenous Areas - 2021',
  'IREG_2021_AUST': 'Indigenous Regions - 2021',
  'LGA_2023_AUST': 'Local Government Areas - 2023',
  'LGA_2022_AUST': 'Local Government Areas - 2022',
  'LGA_2021_AUST': 'Local Government Areas - 2021',
  'SED_2022_AUST': 'State Electoral Divisions - 2022',
  'SED_2021_AUST': 'State Electoral Divisions - 2021',
  'CED_2021_AUST': 'Commonwealth Electoral Divisions - 2021',
  'POA_2021_AUST': 'Postal Areas - 2021',
  'TR_2021_AUST': 'Tourism Regions - 2021',
  'ADD_2021_AUST': 'Australian Drainage Divisions - 2021',
  'SAL_2021_AUST': 'Suburbs and Localities - 2021',
  'MB_DZN_2021_AUST': 'Destination Zones - 2021',
  'DZN_SA2_2021_AUST': 'Destination Zones to Statistical Areas Level 2 - 2021',
  'SUA_2021_AUST': 'Significant Urban Areas - 2021',
  'UCL_SOSR_SOS_2021_AUST': 'Urban Centres and Localities, Section of State and Section of State Range - 2021',
  'SUA_association_2016_2021': 'Significant Urban Area association - 2016 to 2021',
  'UCL_association_2016_2021': 'Urban Centre and Locality association - 2016 to 2021',
  'RA_2021_AUST': 'Remoteness Areas - 2021'
}


In [21]:
dfs = { file.split('.')[0] : pd.read_csv(f'../data/core/mb-alloc/{file}') for file in os.listdir('../data/core/mb-alloc')}

/tmp/ipykernel_56920/2926999295.py:1: DtypeWarning: Columns (1,5,6,8,10,14) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs = { file.split('.')[0] : pd.read_csv(f'../data/core/mb-alloc/{file}', index_col=0) for file in os.listdir('../data/core/mb-alloc')}
/tmp/ipykernel_56920/2926999295.py:1: DtypeWarning: Columns (1,2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs = { file.split('.')[0] : pd.read_csv(f'../data/core/mb-alloc/{file}', index_col=0) for file in os.listdir('../data/core/mb-alloc')}
/tmp/ipykernel_56920/2926999295.py:1: DtypeWarning: Columns (1,2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs = { file.split('.')[0] : pd.read_csv(f'../data/core/mb-alloc/{file}', index_col=0) for file in os.listdir('../data/core/mb-alloc')}
/tmp/ipykernel_56920/2926999295.py:1: DtypeWarning: Columns (1,2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs = { 

In [24]:
for k, df in dfs.items():
    df.to_csv(f'../data/core/mb-alloc/{k}.csv', index=False)
    print(k)

DZN_SA2_2021_AUST
RA_2021_AUST
MB_2021_AUST
LGA_2022_AUST
AUS_2021_AUST
SA3_2021_AUST
SED_2022_AUST
GCCSA_2021_AUST
UCL_association_2016_2021
CED_2021_AUST
UCL_SOSR_SOS_2021_AUST
LGA_2023_AUST
STE_2021_AUST
INDIGENOUS_STRUCTURE_ALLOCATION_2021
ADD_2021_AUST
SA2_2021_AUST
MB_DZN_2021_AUST
SED_2021_AUST
POA_2021_AUST
LGA_2021_AUST
ILOC_2021_AUST
IARE_2021_AUST
TR_2021_AUST
SUA_association_2016_2021
SA1_2021_AUST
SA4_2021_AUST
SUA_2021_AUST
SAL_2021_AUST
IREG_2021_AUST


In [33]:
{k : (len([c for c in df.columns if c in ['MB_CODE_2021', 'SA1_CODE_2021', 'SA2_CODE_2021', 'SA3_CODE_2021', 'SA4_CODE_2021', 'GCCSA_CODE_2021']]), len([c for c in df.columns if c in ['STATE_CODE_2021', 'AUS_CODE_2021']])) for k, df in dfs.items()}

{'DZN_SA2_2021_AUST': (1, 2),
 'RA_2021_AUST': (1, 2),
 'MB_2021_AUST': (6, 2),
 'LGA_2022_AUST': (1, 2),
 'AUS_2021_AUST': (0, 1),
 'SA3_2021_AUST': (3, 2),
 'SED_2022_AUST': (1, 2),
 'GCCSA_2021_AUST': (1, 2),
 'UCL_association_2016_2021': (0, 0),
 'CED_2021_AUST': (1, 2),
 'UCL_SOSR_SOS_2021_AUST': (1, 2),
 'LGA_2023_AUST': (1, 2),
 'STE_2021_AUST': (0, 2),
 'INDIGENOUS_STRUCTURE_ALLOCATION_2021': (1, 2),
 'ADD_2021_AUST': (1, 1),
 'SA2_2021_AUST': (4, 2),
 'MB_DZN_2021_AUST': (2, 2),
 'SED_2021_AUST': (1, 2),
 'POA_2021_AUST': (1, 1),
 'LGA_2021_AUST': (1, 2),
 'ILOC_2021_AUST': (0, 2),
 'IARE_2021_AUST': (0, 2),
 'TR_2021_AUST': (1, 2),
 'SUA_association_2016_2021': (0, 0),
 'SA1_2021_AUST': (5, 2),
 'SA4_2021_AUST': (2, 2),
 'SUA_2021_AUST': (1, 1),
 'SAL_2021_AUST': (1, 2),
 'IREG_2021_AUST': (0, 2)}

In [44]:
{k : (len([c for c in df.columns if c in ['MB_CODE_2021', 'SA1_CODE_2021', 'SA2_CODE_2021', 'SA3_CODE_2021', 'SA4_CODE_2021']]), len([c for c in df.columns if c in ['GCCSA_CODE_2021']])) for k, df in dfs.items()}

{'DZN_SA2_2021_AUST': (1, 0),
 'RA_2021_AUST': (1, 0),
 'MB_2021_AUST': (5, 1),
 'LGA_2022_AUST': (1, 0),
 'AUS_2021_AUST': (0, 0),
 'SA3_2021_AUST': (2, 1),
 'SED_2022_AUST': (1, 0),
 'GCCSA_2021_AUST': (0, 1),
 'UCL_association_2016_2021': (0, 0),
 'CED_2021_AUST': (1, 0),
 'UCL_SOSR_SOS_2021_AUST': (1, 0),
 'LGA_2023_AUST': (1, 0),
 'STE_2021_AUST': (0, 0),
 'INDIGENOUS_STRUCTURE_ALLOCATION_2021': (1, 0),
 'ADD_2021_AUST': (1, 0),
 'SA2_2021_AUST': (3, 1),
 'MB_DZN_2021_AUST': (2, 0),
 'SED_2021_AUST': (1, 0),
 'POA_2021_AUST': (1, 0),
 'LGA_2021_AUST': (1, 0),
 'ILOC_2021_AUST': (0, 0),
 'IARE_2021_AUST': (0, 0),
 'TR_2021_AUST': (1, 0),
 'SUA_association_2016_2021': (0, 0),
 'SA1_2021_AUST': (4, 1),
 'SA4_2021_AUST': (1, 1),
 'SUA_2021_AUST': (1, 0),
 'SAL_2021_AUST': (1, 0),
 'IREG_2021_AUST': (0, 0)}

# Index/Primary key

For mesh-block based tables: MB_CODE_2021 or its higher level ('SA1_CODE_2021', 'SA2_CODE_2021', 'SA3_CODE_2021', 'SA4_CODE_2021')

For indigenous areas: ILOC_2021_AUST -> IARE_2021_AUST -> IREG_2021_AUST. Primary keys: ILOC_CODE_2021 -> IARE_CODE_2021 -> IREG_CODE_2021


In [54]:
dfs['SUA_2021_AUST']['SUA_NAME_2021'].value_counts()

SUA_NAME_2021
Melbourne                                  350
Sydney                                     333
Brisbane                                   235
Perth                                      175
Not in any Significant Urban Area (NSW)    146
                                          ... 
Warwick                                      1
Emerald                                      1
Yeppoon                                      1
Airlie Beach - Cannonvale                    1
Outside Australia                            1
Name: count, Length: 112, dtype: int64

In [55]:
dfs['UCL_SOSR_SOS_2021_AUST']['UCL_NAME_2021'].value_counts()

UCL_NAME_2021
Sydney               10827
Melbourne            10614
Brisbane              5354
Perth (WA)            4603
Adelaide              2829
                     ...  
Badu Island (L)          1
Black River (L)          1
Bluewater (L)            1
Booral (L)               1
Outside Australia        1
Name: count, Length: 1837, dtype: int64